# Imports


In [1]:
using BenchmarkTools
using XLSX
using DataFrames
using BilevelJuMP, HiGHS, Ipopt, SCIP
using BenchmarkTools

Funcion que despues escribe en un excel el ejemplo

In [2]:
function add_to_dataframe_vars_value(df,var_vector::Vector,var_name::String)
    # Insertar una nueva columna al final
    for i in 1:length(var_vector)
        insertcols!(df, ncol(df) + 1,"$var_name: $i" )
    end
    return df
end

add_to_dataframe_vars_value (generic function with 1 method)

In [3]:
function write_response(name,model,x,y,benchmark::BenchmarkTools.Trial)
    # Escribir el nombre del experimento
    println(name)
    # Escribir el resultado de la solucion primal
    p_s=BilevelJuMP.primal_status(model)
    println("Primal Status: $p_s")
    # Escribir el codigo y tipo de terminacion
    t_s=BilevelJuMP.termination_status(model)
    println("Termination Status: $t_s")
    # Objetive value
    o_v=BilevelJuMP.objective_value(model)
    println("Objetive Value $o_v")
    # Lider Vars Value
    l_vars=BilevelJuMP.value.(x)
    println("Lider Vars: $l_vars")
    # Follower Vars Value
    f_vars=BilevelJuMP.value.(y)
    println("Followers Vars Value $f_vars")

    # Extraer los tiempos y otros datos relevantes
    tiempos = resultado.times  # Tiempos en nanosegundos
    min_tiempo = minimum(tiempos) / 1e9  # Tiempo mínimo en segundos
    max_tiempo = maximum(tiempos) / 1e9  # Tiempo máximo en segundos
    promedio_tiempo = mean(tiempos) / 1e9  # Tiempo promedio en segundos
    
    # Extraer el uso de recursos
    num_asignaciones = sum(resultado.allocs)  # Total de asignaciones de memoria
    memoria_usada = maximum(resultado.memory) / (1024^2)  # Memoria máxima usada en MB

    # Crear un DataFrame para almacenar los resultados
    df_resultados = DataFrame(
    Estatus_Primal=p_s,
    Estatus_Terminación=t_s,
    Valor_Objetivo=o_v,
    
    Mínimo=min_tiempo,
    Máximo=max_tiempo,
    Promedio=promedio_tiempo,
    Asignaciones=num_asignaciones,
    Memoria_MB=memoria_usada
    )
    # Añadir el valor de las x_s 
    df_resultados=add_to_dataframe_vars_value(df_resultados,x,"X_s")
    # Anadir el valor de las y_s
    df_resultados=add_to_dataframe_vars_value(df_resultados,y,"Y_s")

    return df_resultados

end

write_response (generic function with 1 method)

In [4]:
function get_optimal(model)
    resultado=@benchmark optimize!(model)
    return resultado,model
end

get_optimal (generic function with 1 method)

Sacar el tiempo y uso de memoria

# Importar los modulos necesarios (BilevelJuMP y los optimizadores)

# Crear el modelo del experimento

In [6]:
model = BilevelModel()

An Abstract JuMP Model
Feasibility problem with:
Variables: 0
Upper Constraints: 0
Lower Constraints: 0
Bilevel Model
Solution method: BilevelJuMP.NoMode{Float64}
No solver attached

In [7]:
BilevelJuMP.@variable(Lower(model), x)
BilevelJuMP.@variable(Upper(model), y)

y

In [8]:
BilevelJuMP.@objective(Upper(model), Min, 3x + y)
BilevelJuMP.@constraints(Upper(model), begin
    x <= 5
    y <= 8
    y >= 0
end)

BilevelJuMP.@objective(Lower(model), Min, -x)
BilevelJuMP.@constraints(Lower(model), begin
    c1,  x +  y <= 8
    c2, 4x +  y >= 8
    c3, 2x +  y <= 13
    c4, 2x - 7y <= 0
end)

(c1 : x + y ≤ 8, c2 : 4 x + y ≥ 8, c3 : 2 x + y ≤ 13, c4 : 2 x - 7 y ≤ 0)

## Using HiGHS Optimizer

In [9]:
BilevelJuMP.set_optimizer(model, HiGHS.Optimizer)

BilevelJuMP.set_mode(model,
    BilevelJuMP.BigMMode(primal_big_M = 100, dual_big_M = 100))

BilevelJuMP.optimize!(model)

BilevelJuMP.objective_value(model)
@assert abs(BilevelJuMP.objective_value(model) - (3 * (7/2 * 8/15) + 8/15)) < 1e-1 # src

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 9e+02]
  Cost   [1e+00, 3e+00]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+02]
Assessing feasibility of MIP using primal feasibility and integrality tolerance of       1e-06
Solution has               num          max          sum
Col     infeasibilities      0            0            0
Integer infeasibilities      0            0            0
Row     infeasibilities      4           13           30
Row     residuals            0            0            0
Attempting to find feasible solution by solving LP for user-supplied values of discrete variables
Coefficient ranges:
  Matrix [1e+00, 9e+02]
  Cost   [1e+00, 3e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+00, 1e+02]
Presolving model
13 rows, 6 cols, 24 nonzeros  0s
Problem status detected on presolve: Infeasible
Model status        : Infeasible
Objective value     :  0.0000000000e+00
HiGHS run time      :      

# Using SOS1 

In [10]:
set_optimizer(model, SCIP.Optimizer)

BilevelJuMP.set_mode(model, BilevelJuMP.SOS1Mode())

optimize!(model)

objective_value(model)

@assert abs(objective_value(model) - (3 * (3.5 * 8/15) + 8/15)) < 1e-1 # src

presolving:
(round 1, fast)       0 del vars, 3 del conss, 0 add conss, 3 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       0 del vars, 3 del conss, 0 add conss, 7 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, fast)       0 del vars, 3 del conss, 0 add conss, 8 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 4, exhaustive) 0 del vars, 6 del conss, 0 add conss, 8 chg bounds, 3 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
   (0.0s) symmetry computation skipped: there exist constraints that cannot be handled by symmetry methods.
presolving (5 rounds: 5 fast, 2 medium, 2 exhaustive):
 0 deleted vars, 6 deleted constraints, 0 added constraints, 8 tightened bounds, 0 added holes, 3 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 10 variables (0 bin, 0 int, 0 impl, 10 cont) and 10 constraints
      4 constraints of type <SOS1>
      6 constrain

# Using Product Mode

In [11]:
set_optimizer(model, Ipopt.Optimizer)

BilevelJuMP.set_mode(model, BilevelJuMP.ProductMode())

optimize!(model)

objective_value(model)

@assert abs(objective_value(model) - (3 * (7/2 * 8/15) + 8/15)) < 1e-1 # src


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        4
Number of nonzeros in inequality constraint Jacobian.:       30
Number of nonzeros in Lagrangian Hessian.............:        8

Total number of variables............................:        6
                     variables with only lower bounds:        1
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        3
Total number of equality constraints.................:        1
Total number of inequality co

In [12]:


resultado=@benchmark optimize!(model)

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        4
Number of nonzeros in inequality constraint Jacobian.:       30
Number of nonzeros in Lagrangian Hessian.............:        8

Total number of variables............................:        6
                     variables with only lower bounds:        1
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        3
Total number of equality constraints.................:        1
Total number of inequality constraints...............:       11
        inequality constraints with only lower bounds:        2
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        9

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 8.00e+00 1.25e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

BenchmarkTools.Trial: 175 samples with 1 evaluation.
 Range (min … max):  22.654 ms … 36.814 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     27.421 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   27.736 ms ±  3.287 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▂  ▂  █▂  ▂▅▃      ▃   ▃   ▆        ▂  ▃                    
  ▇█▄▇██▅███▇███▅██▇▇▁█▇▄██▇▅▇██▅▇▇▇▅▇▅█▁▇█▁█▁▅▇▅▅▁▄▅▅▄▄▄▁▄▁▄ ▄
  22.7 ms         Histogram: frequency by time        35.3 ms <

 Memory estimate: 182.45 KiB, allocs estimate: 3884.

In [13]:
resultado

BenchmarkTools.Trial: 175 samples with 1 evaluation.
 Range (min … max):  22.654 ms … 36.814 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     27.421 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   27.736 ms ±  3.287 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▂  ▂  █▂  ▂▅▃      ▃   ▃   ▆        ▂  ▃                    
  ▇█▄▇██▅███▇███▅██▇▇▁█▇▄██▇▅▇██▅▇▇▇▅▇▅█▁▇█▁█▁▅▇▅▅▁▄▅▅▄▄▄▁▄▁▄ ▄
  22.7 ms         Histogram: frequency by time        35.3 ms <

 Memory estimate: 182.45 KiB, allocs estimate: 3884.

In [28]:
using BenchmarkTools
using XLSX
using DataFrames

function tarea()
    sleep(2)  # Simula una tarea que toma tiempo
    return "Tarea completada"
end

# Ejecutar el benchmark
resultado = @benchmark tarea()

# Extraer los tiempos y otros datos relevantes
tiempos = resultado.times  # Tiempos en nanosegundos
min_tiempo = minimum(tiempos) / 1e9  # Tiempo mínimo en segundos
max_tiempo = maximum(tiempos) / 1e9  # Tiempo máximo en segundos
promedio_tiempo = mean(tiempos) / 1e9  # Tiempo promedio en segundos

# Extraer el uso de recursos
num_asignaciones = sum(resultado.allocs)  # Total de asignaciones de memoria
memoria_usada = maximum(resultado.memory) / (1024^2)  # Memoria máxima usada en MB

# Crear un DataFrame para almacenar los resultados
df_resultados = DataFrame(
    Mínimo=min_tiempo,
    Máximo=max_tiempo,
    Promedio=promedio_tiempo,
    Asignaciones=num_asignaciones,
    Memoria_MB=memoria_usada
)

Row,Mínimo,Máximo,Promedio,Asignaciones,Memoria_MB
,Float64,Float64,Float64,Int64,Float64
1,2.00089,2.00137,2.00106,66,0.00164795


In [30]:
df_resultados

Row,Mínimo,Máximo,Promedio,Asignaciones,Memoria_MB
,Float64,Float64,Float64,Int64,Float64
1,2.00089,2.00137,2.00106,66,0.00164795


In [29]:
# Exportar múltiples tablas o resultados a diferentes hojas
XLSX.openxlsx("resultados_benchmark.xlsx", mode="w") do xf
    XLSX.writetable(xf, "Resultados", df_resultados)
end

MethodError: MethodError: no method matching writetable(::XLSX.XLSXFile, ::String, ::DataFrame)
Closest candidates are:
  writetable(!Matched::Union{AbstractString, IO}, ::Any, ::Any; overwrite, sheetname, anchor_cell) at ~/.julia/packages/XLSX/vzApD/src/write.jl:672
  writetable(!Matched::Union{AbstractString, IO}, ::Any; kw...) at ~/.julia/packages/XLSX/vzApD/src/tables_interface.jl:30

In [17]:
time,model=get_optimal(model)

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        4
Number of nonzeros in inequality constraint Jacobian.:       30
Number of nonzeros in Lagrangian Hessian.............:        8

Total number of variables............................:        6
                     variables with only lower bounds:        1
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        3
Total number of equality constraints.................:        1
Total number of inequality constraints...............:       11
        inequality constraints with only lower bounds:        2
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        9

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 8.00e+00 1.25e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

(Trial(22.831 ms), An Abstract JuMP Model
Minimization problem with:
Variables: 2
Upper objective function type: 
GenericAffExpr{Float64, BilevelVariableRef}
Lower objective function type: 
GenericAffExpr{Float64, BilevelVariableRef}
Upper Constraints: 3
Lower Constraints: 4
Bilevel Model
Solution method: BilevelJuMP.ProductMode{Float64}(0.0, false, 0, nothing)
Solver name: Ipopt
Names registered in the model: c1, c2, c3, c4, x, y)

In [18]:
time

BenchmarkTools.Trial: 186 samples with 1 evaluation.
 Range (min … max):  22.831 ms … 46.769 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     25.814 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   26.948 ms ±  3.750 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▇▂▄█▄▃▇▃▆▃▃▃                                               
  ▃█████████████▄▇█▆▅▆▄▅▃▃▅▅▆▄▃▆▁▃▁▁▁▅▁▃▅▃▁▃▄▃▁▁▁▁▃▁▁▁▁▁▁▁▁▁▄ ▃
  22.8 ms         Histogram: frequency by time        39.3 ms <

 Memory estimate: 182.45 KiB, allocs estimate: 3884.

In [19]:
model

An Abstract JuMP Model
Minimization problem with:
Variables: 2
Upper objective function type: 
GenericAffExpr{Float64, BilevelVariableRef}
Lower objective function type: 
GenericAffExpr{Float64, BilevelVariableRef}
Upper Constraints: 3
Lower Constraints: 4
Bilevel Model
Solution method: BilevelJuMP.ProductMode{Float64}(0.0, false, 0, nothing)
Solver name: Ipopt
Names registered in the model: c1, c2, c3, c4, x, y

In [20]:
typeof(resultado)

BenchmarkTools.Trial

In [24]:
using DataFrames
using XLSX

# Datos de ejemplo
p_s = [1, 2, 3]
t_s = ["Terminado", "En Progreso", "Fallido"]
o_v = [100, 200, 300]
min_tiempo = [10, 20, 30]
max_tiempo = [15, 25, 35]
promedio_tiempo = [12.5, 22.5, 32.5]
num_asignaciones = [5, 10, 15]
memoria_usada = [1000, 2000, 3000]

# Crear el DataFrame inicial
df_resultados = DataFrame(
    Estatus_Primal=p_s,
    Estatus_Terminación=t_s,
    Valor_Objetivo=o_v,
    Mínimo=min_tiempo,
    Máximo=max_tiempo,
    Promedio=promedio_tiempo,
    Asignaciones=num_asignaciones,
    Memoria_MB=memoria_usada
)

# Vector dinámico x
x = [1.1, 2.2, 3.3] # Puede tener tamaño variable

# Agregar columnas al DataFrame
for i in 1:length(x)
    #df_resultados[!, Symbol("columna_$i")] = x[i]
    df_resultados=DataFrame(df_resultados,Symbol("columna_$i")=>x[i])
end

# Guardar el DataFrame en un archivo Excel
nombre_archivo = "resultados.xlsx"

# Crear o abrir el archivo Excel y escribir el DataFrame en la hoja "hola"
XLSX.openxlsx(nombre_archivo, mode="w") do xf
    XLSX.writetable(xf, "hola", collect(DataFrames.eachcol(df_resultados)), DataFrames.names(df_resultados))
end

# Mostrar mensaje de confirmación
println("El DataFrame ha sido guardado en $nombre_archivo en la hoja 'hola'.")

MethodError: MethodError: no method matching DataFrame(::DataFrame, ::Pair{Symbol, Float64})
Closest candidates are:
  DataFrame(!Matched::Pair{Symbol}...; makeunique, copycols) at ~/.julia/packages/DataFrames/kcA9R/src/dataframe/dataframe.jl:257
  DataFrame(::DataFrame; copycols) at ~/.julia/packages/DataFrames/kcA9R/src/dataframe/dataframe.jl:255
  DataFrame(::Any; copycols) at ~/.julia/packages/DataFrames/kcA9R/src/other/tables.jl:48
  ...

In [31]:
using DataFrames

# Crear un DataFrame inicial
df = DataFrame(A = rand(1), B = rand(1))

# Insertar una nueva columna al final
insertcols!(df, ncol(df) + 1, "NuevaColumna" => 100)

println(df)

1×3 DataFrame
 Row │ A         B         NuevaColumna 
     │ Float64   Float64   Int64        
─────┼──────────────────────────────────
   1 │ 0.613313  0.560447           100


In [36]:
print(df)

1×3 DataFrame
 Row │ A         B         NuevaColumna 
     │ Float64   Float64   Int64        
─────┼──────────────────────────────────
   1 │ 0.613313  0.560447           100

In [34]:
# Exportar múltiples tablas o resultados a diferentes hojas
XLSX.openxlsx("ee.xlsx", mode="w") do xf
    XLSX.writetable(xf, "Resultados", df)
end

MethodError: MethodError: no method matching writetable(::XLSX.XLSXFile, ::String, ::DataFrame)
Closest candidates are:
  writetable(!Matched::Union{AbstractString, IO}, ::Any, ::Any; overwrite, sheetname, anchor_cell) at ~/.julia/packages/XLSX/vzApD/src/write.jl:672
  writetable(!Matched::Union{AbstractString, IO}, ::Any; kw...) at ~/.julia/packages/XLSX/vzApD/src/tables_interface.jl:30

In [37]:
using XLSX
using DataFrames

# Crear un DataFrame de ejemplo
df = DataFrame(A = 1:5, B = ["uno", "dos", "tres", "cuatro", "cinco"])

# Abrir o crear un archivo Excel
XLSX.openxlsx("mi_archivo.xlsx", mode="w") do xf
    # Escribir el DataFrame en la hoja "Resultados"
    XLSX.writetable(xf, "Resultados", df)
end

MethodError: MethodError: no method matching writetable(::XLSX.XLSXFile, ::String, ::DataFrame)
Closest candidates are:
  writetable(!Matched::Union{AbstractString, IO}, ::Any, ::Any; overwrite, sheetname, anchor_cell) at ~/.julia/packages/XLSX/vzApD/src/write.jl:672
  writetable(!Matched::Union{AbstractString, IO}, ::Any; kw...) at ~/.julia/packages/XLSX/vzApD/src/tables_interface.jl:30

In [38]:
using XLSX
using DataFrames

# Crear un DataFrame de ejemplo
df = DataFrame(A = 1:5, B = ["uno", "dos", "tres", "cuatro", "cinco"])

# Escribir el DataFrame en un archivo Excel
XLSX.writetable("Resultados.xlsx", "Resultados", df)

MethodError: MethodError: no method matching length(::DataFrame)
Closest candidates are:
  length(!Matched::Union{Base.KeySet, Base.ValueIterator}) at abstractdict.jl:58
  length(!Matched::Union{Tables.AbstractColumns, Tables.AbstractRow}) at ~/.julia/packages/Tables/8p03y/src/Tables.jl:180
  length(!Matched::Union{DataStructures.OrderedRobinDict, DataStructures.RobinDict}) at ~/.julia/packages/DataStructures/95DJa/src/ordered_robin_dict.jl:86
  ...

In [65]:
 import DataFrames, XLSX

 df1 = DataFrames.DataFrame(COL1=[10,20,30], COL2=["Fist", "Sec", "Third"])
 df2 = DataFrames.DataFrame(AA=["aa", "bb"], AB=[10.1, 10.2])
 XLSX.writetable("report.xlsx", "REPORT_m" => df1; overwrite=true )

In [66]:
typeof(df2)

DataFrame

In [47]:
using XLSX
using DataFrames

# Leer la tabla desde el archivo Excel
tabla = XLSX.readtable("report.xlsx","REPORT_m")



XLSX.DataTable(Any[Any[10, 20, 30], Any["Fist", "Sec", "Third"]], [:COL1, :COL2], Dict(:COL2 => 2, :COL1 => 1))

In [49]:
typeof(tabla)

XLSX.DataTable

In [50]:
table=DataFrame(tabla)

Row,COL1,COL2
,Any,Any
1,10,Fist
2,20,Sec
3,30,Third


In [51]:
table

Row,COL1,COL2
,Any,Any
1,10,Fist
2,20,Sec
3,30,Third


In [52]:
push!(table,[3,4])

Row,COL1,COL2
,Any,Any
1,10,Fist
2,20,Sec
3,30,Third
4,3,4


In [54]:
XLSX.writetable("report.xlsx", "REPORT_m" => table; overwrite=true )

In [55]:
using DataFrames

# Crear dos DataFrames de ejemplo
df1 = DataFrame(ID = [1, 2, 3], Nombre = ["A", "B", "C"])
df2 = DataFrame(ID = [4, 5, 6], Nombre = ["D", "E", "F"])

# Unir los DataFrames
df_unido = vcat(df1, df2)

# Mostrar el DataFrame unido
println(df_unido)

6×2 DataFrame
 Row │ ID     Nombre 
     │ Int64  String 
─────┼───────────────
   1 │     1  A
   2 │     2  B
   3 │     3  C
   4 │     4  D
   5 │     5  E
   6 │     6  F


In [67]:
df_=DataFrame(COL1=[22],COL2=[34])

Row,COL1,COL2
,Int64,Int64
1,22,34


In [68]:
println(df_)

1×2 DataFrame
 Row │ COL1   COL2  
     │ Int64  Int64 
─────┼──────────────
   1 │    22     34


In [72]:
typeof(table)

DataFrame

In [69]:
df_unido = vcat(table, df_)

Row,COL1,COL2
,Any,Any
1,10,Fist
2,20,Sec
3,30,Third
4,3,4
5,22,34


In [70]:
println(df_unido)

5×2 DataFrame
 Row │ COL1  COL2  
     │ Any   Any   
─────┼─────────────
   1 │ 10    Fist
   2 │ 20    Sec
   3 │ 30    Third
   4 │ 3     4
   5 │ 22    34


In [71]:
XLSX.writetable("report.xlsx", "REPORT_m" => df_unido; overwrite=true )

In [62]:
# Nombre del archivo a verificar
archivo = "mi_archivo.xlsx"

# Comprobar si el archivo existe
if isfile(archivo)
    println("El archivo existe.")
else
    println("El archivo no existe.")
end

El archivo existe.
